In [2]:
import os
import sys
import shutil
colmap_input_dir = '/kaggle/input/colmap'
colmap_install_dir = '/kaggle/working/colmap'

if not os.path.isdir(colmap_install_dir):
    shutil.copytree(colmap_input_dir, colmap_install_dir)
os.system(f'''cd {colmap_install_dir}/build && \
cmake .. -GNinja -DCMAKE_CUDA_ARCHITECTURES=native && \
ninja && \
ninja install
''')

pycolmap_input_dir = '/kaggle/input/pycolmap'
pycolmap_install_dir = '/kaggle/working/pycolmap'
if not os.path.isdir(pycolmap_install_dir):
    shutil.copytree(pycolmap_input_dir, pycolmap_install_dir)
os.system(f'pip install -e {pycolmap_install_dir}')
if pycolmap_install_dir not in sys.path:
    sys.path.append(pycolmap_install_dir)


-- Found installed version of Eigen: /usr/lib/cmake/eigen3
-- Found required Ceres dependency: Eigen version 3.3.7 in /usr/include/eigen3
-- Found required Ceres dependency: glog
-- Found installed version of gflags: /usr/lib/x86_64-linux-gnu/cmake/gflags
-- Detected gflags version: 2.2.2
-- Found required Ceres dependency: gflags
-- Found Ceres version: 1.14.0 installed in: /usr with components: [EigenSparse, SparseLinearAlgebraLibrary, LAPACK, SuiteSparse, CXSparse, SchurSpecializations, OpenMP, Multithreading]
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found version "1.71.0") found components: program_options filesystem graph system unit_test_framework 
-- Found Eigen
--   Includes : /usr/include/eigen3
-- Found FreeImage
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libfreeimage.so
-- Found FLANN
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libflann.so
-- Found LZ4
--   Includes : /usr/include

    error: subprocess-exited-with-error
    
    × python setup.py develop did not run successfully.
    │ exit code: 1
    ╰─> [44 lines of output]
        running develop
        running egg_info
        writing pycolmap.egg-info/PKG-INFO
        writing dependency_links to pycolmap.egg-info/dependency_links.txt
        writing top-level names to pycolmap.egg-info/top_level.txt
        reading manifest file 'pycolmap.egg-info/SOURCES.txt'
        adding license file 'LICENSE'
        writing manifest file 'pycolmap.egg-info/SOURCES.txt'
        running build_ext
        CMake Error: The current CMakeCache.txt directory /kaggle/working/pycolmap/build/temp.linux-x86_64-3.7/CMakeCache.txt is different than the directory /kaggle/working/inputs/pycolmap/build/temp.linux-x86_64-3.7 where CMakeCache.txt was created. This may result in binaries being created in the wrong place. If you are not sure, reedit the CMakeCache.txt
        CMake Error: The source "/kaggle/working/pycolmap/CMakeLists

  Rolling back uninstall of pycolmap
  Moving to /opt/conda/lib/python3.7/site-packages/pycolmap-0.3.0.dist-info/
   from /opt/conda/lib/python3.7/site-packages/~ycolmap-0.3.0.dist-info
  Moving to /opt/conda/lib/python3.7/site-packages/pycolmap.cpython-37m-x86_64-linux-gnu.so
   from /tmp/pip-uninstall-lkqnorvs/pycolmap.cpython-37m-x86_64-linux-gnu.so
  Moving to /opt/conda/lib/python3.7/site-packages/pycolmap.libs/
   from /opt/conda/lib/python3.7/site-packages/~ycolmap.libs


In [ ]:
!pip install -e /kaggle/working/pycolmap

Obtaining file:///kaggle/working/pycolmap
  Preparing metadata (setup.py) ... done


In [7]:
# General utilities
import os, sys
from tqdm import tqdm
from time import time
from fastprogress import progress_bar
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from dataclasses import dataclass, asdict, field
from typing import Optional, Tuple, List
import traceback
import subprocess
from pprint import pprint
import shutil
from pathlib import Path
from pprint import pprint
from functools import lru_cache
from matplotlib import pyplot as plt
from glob import glob


# CV/ML
import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from torch import multiprocessing as mp


def internet_available(host="8.8.8.8", port=53, timeout=1):
    # https://stackoverflow.com/a/33117579
    import socket
    try:
        socket.setdefaulttimeout(timeout)
        socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
        return True
    except socket.error as ex:
        return False

    
def run_sh(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())
    
    
def install_from_input_directory(dirname):
    src_dir = f'/kaggle/input/{dirname}/'
    install_dir = os.path.realpath(f'./{dirname}/')
    assert os.path.isdir(src_dir)
    if not os.path.isdir(install_dir):
        shutil.copytree(src_dir, install_dir)

    if dirname.endswith('silk'):
        if os.path.isfile(f'{install_dir}/silk'):
            os.remove(f'{install_dir}/silk')
        if not os.path.isdir(f'{install_dir}/silk'):
            shutil.copytree(f'{install_dir}/lib', f'{install_dir}/silk')
            
    os.system(f'pip install -e {install_dir}')
    if install_dir not in sys.path:
        sys.path.append(install_dir)
    
    
def install_hloc_local_features():
    weights = {
        'openibl': {
            'src': '/kaggle/input/hloc-local-feature-weights/vgg16_netvlad.pth',
            'dst': '/root/.cache/torch/hub/checkpoints/vgg16_netvlad.pth',
        },
        'netvlad': {
            'src': '/kaggle/input/hloc-local-feature-weights/Pitts30K_struct.mat',
            'dst': '/root/.cache/torch/hub/netvlad/VGG16-NetVLAD-Pitts30K.mat'
        },
        'openibl_src': {
            'src': '/kaggle/input/openibl-zipball/yxgeee-OpenIBL-f3ef4fb',
            'dst': '/root/.cache/torch/hub/yxgeee_OpenIBL_master'
        }
    }
    for alg in weights.keys():
        print('installing', alg)
        w = weights[alg]
        if os.path.isfile(w['src']):
            os.makedirs(os.path.dirname(w['dst']), exist_ok=True)
            shutil.copyfile(w['src'], w['dst'])
        elif os.path.isdir(w['src']):
            if os.path.isdir(w['dst']):
                shutil.rmtree(w['dst'])
            shutil.copytree(w['src'], w['dst'])
            
def install_pycolmap():
    colmap_input_dir = '/kaggle/input/colmap'
    colmap_install_dir = '/kaggle/working/colmap'

    if not os.path.isdir(colmap_install_dir):
        shutil.copytree(colmap_input_dir, colmap_install_dir)
    os.system(f'''cd {colmap_install_dir}/build && \
    cmake .. -GNinja -DCMAKE_CUDA_ARCHITECTURES=all-major && \
    ninja && \
    ninja install
    ''')

    pycolmap_input_dir = '/kaggle/input/pycolmap'
    pycolmap_install_dir = '/kaggle/working/pycolmap'
    if not os.path.isdir(pycolmap_install_dir):
        shutil.copytree(pycolmap_input_dir, pycolmap_install_dir)
    os.system(f'pip install -e {pycolmap_install_dir}')
    if pycolmap_install_dir not in sys.path:
        sys.path.append(pycolmap_install_dir)
            
# install custom colmap build
install_pycolmap()
import pycolmap

            
# install dependencies for SiLK
!pip install --no-deps \
    /kaggle/input/omegaconf222py3/antlr4_python3_runtime-4.9.3-py3-none-any.whl \
    /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl \
    /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl \
    /kaggle/input/loguru006py3/loguru-0.6.0-py3-none-any.whl


install_from_input_directory('facebookresearch-silk')

from silk.backbones.superpoint.vgg import ParametricVGG
from silk.backbones.silk.silk import SiLKVGG as SiLK
from silk.backbones.silk.silk import from_feature_coords_to_image_coords
from silk.config.model import load_model_from_checkpoint
from silk.models.silk import matcher


# install hloc and weights
install_from_input_directory('hierarchical-localization-my')
install_hloc_local_features()

from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
from hloc import extract_features as hloc_extract_features
from hloc import pairs_from_retrieval as hloc_pairs_from_retrieval

CMake Error: The current CMakeCache.txt directory /kaggle/working/colmap/build/CMakeCache.txt is different than the directory /kaggle/working/inputs/colmap/build where CMakeCache.txt was created. This may result in binaries being created in the wrong place. If you are not sure, reedit the CMakeCache.txt
CMake Error: The source "/kaggle/working/colmap/CMakeLists.txt" does not match the source "/kaggle/working/inputs/colmap/CMakeLists.txt" used to generate cache.  Re-run cmake with a different source directory.


Obtaining file:///kaggle/working/pycolmap
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: pycolmap
    Found existing installation: pycolmap 0.3.0
    Uninstalling pycolmap-0.3.0:
      Successfully uninstalled pycolmap-0.3.0
  Running setup.py develop for pycolmap


    error: subprocess-exited-with-error
    
    × python setup.py develop did not run successfully.
    │ exit code: 1
    ╰─> [44 lines of output]
        running develop
        running egg_info
        writing pycolmap.egg-info/PKG-INFO
        writing dependency_links to pycolmap.egg-info/dependency_links.txt
        writing top-level names to pycolmap.egg-info/top_level.txt
        reading manifest file 'pycolmap.egg-info/SOURCES.txt'
        adding license file 'LICENSE'
        writing manifest file 'pycolmap.egg-info/SOURCES.txt'
        running build_ext
        CMake Error: The current CMakeCache.txt directory /kaggle/working/pycolmap/build/temp.linux-x86_64-3.7/CMakeCache.txt is different than the directory /kaggle/working/inputs/pycolmap/build/temp.linux-x86_64-3.7 where CMakeCache.txt was created. This may result in binaries being created in the wrong place. If you are not sure, reedit the CMakeCache.txt
        CMake Error: The source "/kaggle/working/pycolmap/CMakeLists

  Rolling back uninstall of pycolmap
  Moving to /opt/conda/lib/python3.7/site-packages/pycolmap-0.3.0.dist-info/
   from /opt/conda/lib/python3.7/site-packages/~ycolmap-0.3.0.dist-info
  Moving to /opt/conda/lib/python3.7/site-packages/pycolmap.cpython-37m-x86_64-linux-gnu.so
   from /tmp/pip-uninstall-v157_5g6/pycolmap.cpython-37m-x86_64-linux-gnu.so
  Moving to /opt/conda/lib/python3.7/site-packages/pycolmap.libs/
   from /opt/conda/lib/python3.7/site-packages/~ycolmap.libs
Processing /kaggle/input/omegaconf222py3/antlr4_python3_runtime-4.9.3-py3-none-any.whl
Processing /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl
Processing /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl
Processing /kaggle/input/loguru006py3/loguru-0.6.0-py3-none-any.whl
antlr4-python3-runtime is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
omegaconf is already installed with the same version as the prov

ERROR: Could not find a version that satisfies the requirement torch==1.11.0+cu113 (from silk-keypoint-library) (from versions: 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1)
ERROR: No matching distribution found for torch==1.11.0+cu113


Obtaining file:///kaggle/working/hierarchical-localization-my
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: hloc
    Found existing installation: hloc 1.3
    Uninstalling hloc-1.3:
      Successfully uninstalled hloc-1.3
  Running setup.py develop for hloc
installing openibl


installing netvlad
installing openibl_src


In [3]:
import torch

In [4]:
torch.cuda.is_available()

True